In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("amazon/Reviews.csv")

In [3]:
amzn = data[["Score", "Text"]]
amzn.columns = ["stars", "text"]
amzn["source"] = "Amazon"

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
amzn.head()

,stars,text,source
0,5,I have bought several of the Vitality canned d...,Amazon
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon
2,4,This is a confection that has been around a fe...,Amazon
3,2,If you are looking for the secret ingredient i...,Amazon
4,5,Great taffy at a great price. There was a wid...,Amazon


In [5]:
data = pd.read_csv("yelp.csv")

In [6]:
yelp = data[["stars", "text"]]
yelp["source"] = "Yelp"

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
yelp.head()

,stars,text,source
0,5,My wife took me here on my birthday for breakf...,Yelp
1,5,I have no idea why some people give bad review...,Yelp
2,4,love the gyro plate. Rice is so good and I als...,Yelp
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",Yelp
4,5,General Manager Scott Petello is a good egg!!!...,Yelp


In [8]:
df = amzn.append(yelp).reset_index(drop = True)

In [9]:
display(df)

,stars,text,source
0,5,I have bought several of the Vitality canned d...,Amazon
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon
2,4,This is a confection that has been around a fe...,Amazon
3,2,If you are looking for the secret ingredient i...,Amazon
4,5,Great taffy at a great price. There was a wid...,Amazon
...,...,...,...
578449,3,First visit...Had lunch here today - used my G...,Yelp
578450,4,Should be called house of deliciousness!\n\nI ...,Yelp
578451,4,I recently visited Olive and Ivy for business ...,Yelp
578452,2,My nephew just moved to Scottsdale recently so...,Yelp


In [10]:
from nltk import wordpunct_tokenize, word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [11]:
tokens = df['text'].apply(lambda x: wordpunct_tokenize(x))
df['text'] = tokens

In [12]:
mini = df.iloc[0:10,:]

In [13]:
mini

,stars,text,source
0,5,"[I, have, bought, several, of, the, Vitality, ...",Amazon
1,1,"[Product, arrived, labeled, as, Jumbo, Salted,...",Amazon
2,4,"[This, is, a, confection, that, has, been, aro...",Amazon
3,2,"[If, you, are, looking, for, the, secret, ingr...",Amazon
4,5,"[Great, taffy, at, a, great, price, ., There, ...",Amazon
5,4,"[I, got, a, wild, hair, for, taffy, and, order...",Amazon
6,5,"[This, saltwater, taffy, had, great, flavors, ...",Amazon
7,5,"[This, taffy, is, so, good, ., It, is, very, s...",Amazon
8,5,"[Right, now, I, ', m, mostly, just, sprouting,...",Amazon
9,5,"[This, is, a, very, healthy, dog, food, ., Goo...",Amazon


In [14]:
sw = stopwords.words('english')

In [15]:
mini['text'] = mini['text'].apply(lambda x: [word for word in x if word not in sw])

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in sw])

In [20]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [21]:
sid = SentimentIntensityAnalyzer()

In [24]:
df['scores'] = df['text'].apply(lambda x: sid.polarity_scores(" ".join(x)))

In [31]:
df.scores.neg.head()

AttributeError: 'Series' object has no attribute 'neg'